# **Code for team project: Abdominal Injury Detection**
 Using open-source dataset(s) and code(s).

In [1]:
# import packages
import os
import pickle
from tqdm.notebook import tqdm
import random
from tabulate import tabulate

import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import pydicom
import matplotlib.pyplot as plt
import torchvision.transforms.v2 as t

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from torch.utils.data import Dataset, DataLoader, Subset
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim import Adam
from torchvision import models
from torchvision.transforms.v2 import Resize, Compose, RandomHorizontalFlip, ColorJitter, RandomAffine, RandomErasing, ToTensor

/opt/conda/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

In [2]:
TRAIN_IMG_PATH = '/kaggle/input/rsna-2023-atd-reduced-256-5mm/reduced_256_tickness_5'
TEST_IMG_PATH = '/kaggle/input/rsna-2023-abdominal-trauma-detection/test_images'
TRAIN_DF_PATH = '/kaggle/input/rsna-2023-abdominal-trauma-detection/train.csv'

In [3]:
def fetch_img_paths(train_img_path):
    img_paths = []
    
    print('Scanning directories...')
    for patient in tqdm(os.listdir(train_img_path)):
        for scan in os.listdir(os.path.join(TRAIN_IMG_PATH, patient)):
            scans = []
            for img in os.listdir(os.path.join(TRAIN_IMG_PATH, patient, scan)):
                scans.append(os.path.join(TRAIN_IMG_PATH, patient, scan, img))
            
            img_paths.append(scans)
            
    return img_paths

In [4]:
def select_elements_with_spacing(input_list, spacing):
    
    """
    Selects elements with a specified spacing from a given list.

    Args:
        input_list (list): The input list from which elements will be selected.
        spacing (int): The spacing between selected elements.

    Returns:
        list: A list of selected elements from the input list.

    Raises:
        ValueError: If the input list does not contain at least 4 * spacing elements.
    """
 
    if len(input_list) < spacing * 4:
        raise ValueError("List should contain at least 4 * spacing elements.")
        
        
    # We want to select elements in the middle part of the abdomen
    lower_bound = int(len(input_list) * 0.4)
    upper_bound = int(len(input_list) * 0.6)

    spacing = (upper_bound - lower_bound) // 3
    
    # start_index = random.randint(lower_bound, upper_bound)
    
    selected_indices = [lower_bound, lower_bound + spacing, lower_bound + (2*spacing), upper_bound]
    
    selected_elements = [input_list[index] for index in selected_indices]
    
    return selected_elements

In [5]:
def standardize_pixel_array(dicom_image):
    """
    Standardizes a DICOM pixel array by applying various transformations.
    
    Args:
        dicom_path (str): Path to the DICOM image file.
        
    Returns:
        np.ndarray: The standardized pixel array of the DICOM image.
    """
    pixel_array = dicom_image.pixel_array
    
    if dicom_image.PixelRepresentation == 1:
        bit_shift = dicom_image.BitsAllocated - dicom_image.BitsStored
        dtype = pixel_array.dtype 
        new_array = (pixel_array << bit_shift).astype(dtype) >>  bit_shift
        pixel_array = pydicom.pixel_data_handlers.util.apply_modality_lut(new_array, dicom_image)

    if dicom_image.PhotometricInterpretation == "MONOCHROME1":
        pixel_array = 1 - pixel_array

    # transform to hounsfield units
    intercept = dicom_image.RescaleIntercept
    slope = dicom_image.RescaleSlope
    pixel_array = pixel_array * slope + intercept

    # windowing
    window_center = int(dicom_image.WindowCenter)
    window_width = int(dicom_image.WindowWidth)
    img_min = window_center - window_width // 2
    img_max = window_center + window_width // 2
    pixel_array = pixel_array.copy()
    pixel_array[pixel_array < img_min] = img_min
    pixel_array[pixel_array > img_max] = img_max

    # normalization
    if pixel_array.max() == pixel_array.min():
        pixel_array = np.zeros_like(pixel_array)  # Handle case of constant array
    else:
        pixel_array = (pixel_array - pixel_array.min()) / (pixel_array.max() - pixel_array.min())

    return pixel_array

In [6]:
def preprocess_jpeg(jpeg_path):
    
    img = cv2.imread(jpeg_path)
    greyscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)/255
    
    return greyscale

In [7]:
# dataset
class AbdominalData(Dataset):
    """
    Custom dataset class for handling abdominal trauma data classification.
    
    Args:
        df_path (str): Path to the CSV file containing patient labels.
        current_fold (int): The current fold for cross-validation.
        num_fold (int, optional): Total number of folds for cross-validation. Default is 5.
    """
    
    def __init__(self, df_path, current_fold, num_fold = 5):
        
        super().__init__()
        
        # collect all the image instance paths
        self.img_paths = fetch_img_paths(TRAIN_IMG_PATH)
                
        self.df = pd.read_csv(df_path)
        
        self.num_fold = num_fold
        self.current_fold = current_fold
        self.kf = KFold(n_splits=num_fold)
        
        self.transform = Compose([
#                             Resize((256, 256), antialias=True),
                            RandomHorizontalFlip(),  # Randomly flip images left-right
                            ColorJitter(brightness=0.2),  # Randomly adjust brightness
                            ColorJitter(contrast=0.2),  # Randomly adjust contrast
                            RandomAffine(degrees=0, shear=10),  # Apply shear transformation
                            RandomAffine(degrees=0, scale=(0.8, 1.2)),  # Apply zoom transformation
                            RandomErasing(p=0.2, scale=(0.02, 0.2)), # Coarse dropout
                            ToTensor(),
                        ])
    
    def __len__(self):
        """
        Returns the total number of samples in the dataset.
        """
        
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        """
        Retrieves a sample from the dataset by index.
        
        Args:
            idx (int): Index of the dataset to retrieve.
        
        Returns:
            dict: A dictionary containing the image data and labels for different abdominal structures.
        """
        
        # sample 4 image instances
        dicom_images = select_elements_with_spacing(self.img_paths[idx],
                                                    spacing = 2)
        patient_id = dicom_images[0].split('/')[-3]
        images = []
        
        for d in dicom_images:
            image = preprocess_jpeg(d)
            images.append(image)
            
        images = np.stack(images)
        image = torch.tensor(images, dtype = torch.float).unsqueeze(dim = 1)
        
        image = self.transform(image).squeeze(dim = 1)
        
        label = self.df[self.df.patient_id == int(patient_id)].values[0][1:-1]
        
        # labels
        bowel = np.argmax(label[0:2], keepdims = True)
        extravasation = np.argmax(label[2:4], keepdims = True)
        kidney = np.argmax(label[4:7], keepdims = False)
        liver = np.argmax(label[7:10], keepdims = False)
        spleen = np.argmax(label[10:], keepdims = False)
        
        
        return {
            'image': image,
            'bowel': bowel,
            'extravasation': extravasation,
            'kidney': kidney,
            'liver': liver,
            'spleen': spleen,
            'patient': int(patient_id) #added
        }
    
    def get_splits(self):
        """
        Splits the dataset into training and validation subsets based on the current fold.
        
        Returns:
            tuple: A tuple containing the training and validation subsets.
        """
        
        fold_data = list(self.kf.split(self.img_paths))
        train_indices, val_indices = fold_data[self.current_fold]

        train_data = self._get_subset(train_indices)
        val_data = self._get_subset(val_indices)
        
        return train_data, val_data

    def _get_subset(self, indices):
        """
        Returns a subset of the dataset based on the provided indices.
        
        Args:
            indices (list): List of indices to include in the subset.
        
        Returns:
            Subset: A subset of the dataset.
        """
        return Subset(self, indices)

In [8]:
class MetricsCalculator:
    
    def __init__(self, mode = 'binary'):
        
        self.probabilities = []
        self.predictions = []
        self.targets = []
        
        self.mode = mode
    
    def update(self, logits, target):
        """
        Update the metrics calculator with predicted values and corresponding targets.
        
        Args:
            predicted (torch.Tensor): Predicted values.
            target (torch.Tensor): Ground truth targets.
        """
        if self.mode == 'binary':
            probabilities = torch.sigmoid(logits)
            predicted = (probabilities > 0.5)
        else:
            probabilities = F.softmax(logits, dim = 1)
            predicted = torch.argmax(probabilities, dim=1)
            
        self.probabilities.extend(probabilities.detach().cpu().numpy())
        self.predictions.extend(predicted.detach().cpu().numpy())
        self.targets.extend(target.detach().cpu().numpy())
    
    def reset(self):
        """Reset the stored predictions and targets."""
        
        self.probabilities = []
        self.predictions = []
        self.targets = []
    
    def compute_accuracy(self):
        """
        Compute the accuracy metric.
        
        Returns:
            float: Accuracy.
        """
        return accuracy_score(self.targets, self.predictions)
    
    def compute_auc(self):
        """
        Compute the AUC (Area Under the Curve) metric.
        
        Returns:
            float: AUC.
        """
        if self.mode == 'multi':
            return roc_auc_score(self.targets, self.probabilities, multi_class = 'ovo', labels=[0, 1, 2])
    
        else:
            return roc_auc_score(self.targets, self.probabilities)

In [9]:
# Model Architecure
class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.input = nn.Conv2d(4, 3, kernel_size = 3)
        model = models.efficientnet_b0(weights = 'IMAGENET1K_V1')
        
        self.features = model.features
        self.avgpool = model.avgpool
        
        self.bowel = nn.Linear(1280, 1)
        self.extravasation = nn.Linear(1280, 1)
        self.kidney = nn.Linear(1280, 3)
        self.liver = nn.Linear(1280,3) 
        self.spleen = nn.Linear(1280, 3)
    
    def forward(self, x):
        
        # extract features
        x = self.input(x)
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        
        # output logits
        bowel = self.bowel(x)
        extravsation = self.extravasation(x)
        kidney = self.kidney(x)
        liver = self.liver(x)
        spleen = self.spleen(x)
        
        return bowel, extravsation, kidney, liver, spleen

In [10]:
model = CNNModel().to('cuda')

In [11]:
BATCH_SIZE = 16
NUM_EPOCHS = 12 #or 10
LR = 1e-4

In [12]:
train_data_0, val_data_0 = AbdominalData('/kaggle/input/rsna-2023-abdominal-trauma-detection/train.csv', current_fold=0).get_splits()
train_data_1, val_data_1 = AbdominalData('/kaggle/input/rsna-2023-abdominal-trauma-detection/train.csv', current_fold=1).get_splits()
train_data_2, val_data_2 = AbdominalData('/kaggle/input/rsna-2023-abdominal-trauma-detection/train.csv', current_fold=2).get_splits()
train_data_3, val_data_3 = AbdominalData('/kaggle/input/rsna-2023-abdominal-trauma-detection/train.csv', current_fold=3).get_splits()
train_data_4, val_data_4 = AbdominalData('/kaggle/input/rsna-2023-abdominal-trauma-detection/train.csv', current_fold=4).get_splits()

train_dataloader_0 = DataLoader(train_data_0,batch_size = BATCH_SIZE, shuffle = True)
val_dataloader_0 = DataLoader(val_data_0,batch_size = BATCH_SIZE, shuffle = False)
train_dataloader_1 = DataLoader(train_data_1,batch_size = BATCH_SIZE, shuffle = True)
val_dataloader_1 = DataLoader(val_data_1,batch_size = BATCH_SIZE, shuffle = False)
train_dataloader_2 = DataLoader(train_data_2,batch_size = BATCH_SIZE, shuffle = True)
val_dataloader_2 = DataLoader(val_data_2,batch_size = BATCH_SIZE, shuffle = False)
train_dataloader_3 = DataLoader(train_data_3,batch_size = BATCH_SIZE, shuffle = True)
val_dataloader_3 = DataLoader(val_data_3,batch_size = BATCH_SIZE, shuffle = False)
train_dataloader_4 = DataLoader(train_data_4,batch_size = BATCH_SIZE, shuffle = True)
val_dataloader_4 = DataLoader(val_data_4,batch_size = BATCH_SIZE, shuffle = False)

Scanning directories...


  0%|          | 0/3147 [00:00<?, ?it/s]

Scanning directories...


/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `transforms.Compose([transforms.ToImageTensor(), transforms.ConvertImageDtype()])`.
  warnings.warn(


  0%|          | 0/3147 [00:00<?, ?it/s]

...

  0%|          | 0/3147 [00:00<?, ?it/s]

Scanning directories...


/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `transforms.Compose([transforms.ToImageTensor(), transforms.ConvertImageDtype()])`.
  warnings.warn(


  0%|          | 0/3147 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:41: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `transforms.Compose([transforms.ToImageTensor(), transforms.ConvertImageDtype()])`.
  warnings.warn(


In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr = LR)
bce_b = nn.BCEWithLogitsLoss(pos_weight = torch.tensor([2.0]).to('cuda'))
bce_e = nn.BCEWithLogitsLoss(pos_weight = torch.tensor([4.0]).to('cuda'))
cce = nn.CrossEntropyLoss(label_smoothing = 0.05, weight = torch.tensor([1.0, 2.0, 4.0]).to('cuda'))
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5, verbose=True)

In [14]:
# initialize metrics objects
train_acc_bowel = MetricsCalculator('binary')
train_acc_extravasation = MetricsCalculator('binary')
train_acc_liver = MetricsCalculator('multi')
train_acc_kidney = MetricsCalculator('multi')
train_acc_spleen = MetricsCalculator('multi')

val_acc_bowel = MetricsCalculator('binary')
val_acc_extravasation = MetricsCalculator('binary')
val_acc_liver = MetricsCalculator('multi')
val_acc_kidney = MetricsCalculator('multi')
val_acc_spleen = MetricsCalculator('multi')

In [15]:
prev_val_best_loss = float('inf')

dataloaders = [(train_dataloader_0, val_dataloader_0),
               (train_dataloader_1, val_dataloader_1),
               (train_dataloader_2, val_dataloader_2), 
               (train_dataloader_3, val_dataloader_3)] #(train_dataloader_4, val_dataloader_4) for 5 fold training

for epoch in range(NUM_EPOCHS):
    
    # training
    model.train()
    
    train_loss = 0.0
    val_loss = 0.0
    
    print(f'Epoch: [{epoch+1}/{NUM_EPOCHS}]')
    
    train_dataloader, val_dataloader = dataloaders[epoch%4] #5 for 5 fold
    
    print(f'Fold: {epoch%4}') #5 for 5 fold
    
    for batch_idx, batch_data in enumerate(tqdm(train_dataloader)):
        
        inputs = batch_data['image'].to('cuda')
        bowel = batch_data['bowel'].to('cuda')
        extravasation = batch_data['extravasation'].to('cuda')
        liver = batch_data['liver'].to('cuda')
        kidney = batch_data['kidney'].to('cuda')
        spleen = batch_data['spleen'].to('cuda')
        
        optimizer.zero_grad()
        b, e, k, l, s = model(inputs)
        b_loss = bce_b(b, bowel.float())
        e_loss = bce_e(e, extravasation.float())
        l_loss = cce(l, liver)
        k_loss = cce(k, kidney)
        s_loss = cce(s, spleen)
        
        total_loss = b_loss + e_loss + l_loss + k_loss + s_loss
        total_loss.backward()
        
        optimizer.step()
        
        # calculate training metrics
        train_loss += total_loss.item()
        train_acc_bowel.update(b, bowel)
        train_acc_extravasation.update(e, extravasation)
        train_acc_liver.update(l, liver)
        train_acc_kidney.update(k, kidney)
        train_acc_spleen.update(s, spleen)
    
    train_loss = train_loss/(batch_idx+1)
    
    # validation
    model.eval()
    running_loss = 0.0
    
    for batch_idx, batch_data in enumerate(tqdm(val_dataloader)):
                                                
        inputs = batch_data['image'].to('cuda')
        bowel = batch_data['bowel'].to('cuda')
        extravasation = batch_data['extravasation'].to('cuda')
        liver = batch_data['liver'].to('cuda')
        kidney = batch_data['kidney'].to('cuda')
        spleen = batch_data['spleen'].to('cuda')

        
        b, e, k, l, s = model(inputs)
        b_loss = bce_b(b, bowel.float())
        e_loss = bce_e(e, extravasation.float())
        l_loss = cce(l, liver)
        k_loss = cce(k, kidney)
        s_loss = cce(s, spleen)
        
        total_loss = b_loss + e_loss + l_loss + k_loss + s_loss
        
        # calculate validation metrics
        val_loss += total_loss.item()
        val_acc_bowel.update(b, bowel)
        val_acc_extravasation.update(e, extravasation)
        val_acc_liver.update(l, liver)
        val_acc_kidney.update(k, kidney)
        val_acc_spleen.update(s, spleen)
    
    
    val_loss = val_loss/(batch_idx+1)
    scheduler.step(val_loss)
    
    if val_loss < prev_val_best_loss:
        prev_val_best_loss = val_loss
        print("Validation Loss improved, Saving Model...")
        torch.save(model, f'efficientnet_b0_{val_loss:.3f}.pth')
        torch.save(model.state_dict(), f'stdi_efficientnet_b0_{val_loss:.3f}')
    
    
    
    # accuracy and auc data
    metrics_data = [
                    ["Bowel", 
                        train_acc_bowel.compute_accuracy(),
                        val_acc_bowel.compute_accuracy(),
                        train_acc_bowel.compute_auc(),
                        val_acc_bowel.compute_auc()],
                    ["Extravasation", 
                        train_acc_extravasation.compute_accuracy(),
                        val_acc_extravasation.compute_accuracy(),
                        train_acc_extravasation.compute_auc(),
                        val_acc_extravasation.compute_auc()],
                    ["Liver", 
                        train_acc_liver.compute_accuracy(),
                        val_acc_liver.compute_accuracy(),
                        train_acc_liver.compute_auc(),
                        val_acc_liver.compute_auc()],
                    ["Kidney", 
                        train_acc_kidney.compute_accuracy(),
                        val_acc_kidney.compute_accuracy(),
                        train_acc_kidney.compute_auc(),
                        val_acc_kidney.compute_auc()],
                    ["Spleen", 
                        train_acc_spleen.compute_accuracy(),
                        val_acc_spleen.compute_accuracy(),
                        train_acc_spleen.compute_auc(),
                        val_acc_spleen.compute_auc()]
                ]
    
    # verbose
    print('')
    print(tabulate(metrics_data, headers=["", "Train Acc", "Val Acc", "Train AUC", "Val AUC"]))
    
    print(f'\nMean Train Loss: {train_loss:.3f}')
    print(f'Mean Val Loss: {val_loss:.3f}\n')
    
    #reset metrics
    train_acc_bowel.reset()
    train_acc_extravasation.reset()
    train_acc_liver.reset()
    train_acc_kidney.reset()
    train_acc_spleen.reset()
    val_acc_bowel.reset()
    val_acc_extravasation.reset()
    val_acc_liver.reset()
    val_acc_kidney.reset()
    val_acc_spleen.reset()


Epoch: [1/12]
Fold: 0


  0%|          | 0/236 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

Validation Loss improved, Saving Model...

                 Train Acc    Val Acc    Train AUC    Val AUC
-------------  -----------  ---------  -----------  ---------
Bowel             0.96603    0.978791     0.582887   0.552871
Extravasation     0.920117   0.942736     0.531829   0.522893
Liver             0.890658   0.903499     0.503382   0.485123
Kidney            0.918259   0.945917     0.53483    0.520016
Spleen            0.856953   0.88017      0.53859    0.597423

Mean Train Loss: 3.354
Mean Val Loss: 3.092

Epoch: [2/12]
Fold: 1


  0%|          | 0/236 [00:00<?, ?it/s]

...

  0%|          | 0/59 [00:00<?, ?it/s]

Validation Loss improved, Saving Model...

                 Train Acc    Val Acc    Train AUC    Val AUC
-------------  -----------  ---------  -----------  ---------
Bowel             0.977978   0.975584     0.887998   0.927473
Extravasation     0.929955   0.954352     0.876916   0.930937
Liver             0.906341   0.898089     0.781904   0.821857
Kidney            0.939772   0.94586      0.833034   0.816676
Spleen            0.887769   0.857749     0.793051   0.854666

Mean Train Loss: 2.603
Mean Val Loss: 2.430

Epoch: [12/12]
Fold: 3


  0%|          | 0/236 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]


                 Train Acc    Val Acc    Train AUC    Val AUC
-------------  -----------  ---------  -----------  ---------
Bowel             0.97904    0.977707     0.865491   0.891703
Extravasation     0.950385   0.914013     0.907993   0.90251
Liver             0.907402   0.91189      0.831555   0.780218
Kidney            0.944282   0.951168     0.82882    0.888153
Spleen            0.889361   0.858811     0.819259   0.795328

Mean Train Loss: 2.494
Mean Val Loss: 2.581



In [53]:
columns = ['Patient', 'Bowel', 'Extravasation', 'Kidney', 'Kidney_low', 'Kidney_high', 'Liver', 'Liver_low', 'Liver_high', 'Spleen','Spleen_low', 'Spleen_high']
predictions_df = pd.DataFrame(columns=columns)

model = CNNModel().to('cuda')

# Load the saved weights
model_path = '/kaggle/working/stdi_efficientnet_b0_2.430'
model.load_state_dict(torch.load(model_path))
model.eval()

for batch_data in (val_dataloader_4):
    inputs = batch_data['image'].to('cuda')
    bowel = batch_data['bowel'].to('cuda')
    extravasation = batch_data['extravasation'].to('cuda')
    liver = batch_data['liver'].to('cuda')
    kidney = batch_data['kidney'].to('cuda')
    spleen = batch_data['spleen'].to('cuda')
    
    patients = batch_data['patient'].tolist()

    b, e, k, l, s = model(inputs)

    bowel_probs = torch.sigmoid(b.detach())
    extravasation_probs = torch.sigmoid(e.detach())
    
    kidney_probs = F.softmax(k.detach(), dim=1)
    liver_probs = F.softmax(l.detach(), dim=1)
    spleen_probs = F.softmax(s.detach(), dim=1)
  
    bowel_list=bowel_probs.tolist()
    extravasation_list = extravasation_probs.tolist()
    kidney_list = kidney_probs.tolist()
    liver_list = liver_probs.tolist()
    spleen_list = spleen_probs.tolist()
    
    result_dicts=[]
    for i in range(len(patients)):
        result_dict = {
            'Patient': patients[i],
            'Bowel': bowel_list[i][0],
            'Extravasation': extravasation_list[i][0],
            'Kidney': kidney_list[i][0],
            'Kidney_low': kidney_list[i][1],
            'Kidney_high': kidney_list[i][2],
            'Liver': liver_list[i][0],
            'Liver_low': liver_list[i][1],
            'Liver_high': liver_list[i][2],
            'Spleen': spleen_list[i][0],
            'Spleen_low': spleen_list[i][1],
            'Spleen_high': spleen_list[i][2]}
        result_dicts.append(result_dict)
        #predictions_df = predictions_df.append(result_dict, ignore_index=True)
# Concatenate the list of result dictionaries into a DataFrame

    result_dfs = [pd.DataFrame(result_dict, index=[0]) for result_dict in result_dicts]

# Concatenate the existing DataFrame with the new results
    predictions_df = pd.concat([predictions_df] + result_dfs, ignore_index=True)
        



In [54]:
predictions_df

,Patient,Bowel,Extravasation,Kidney,Kidney_low,Kidney_high,Liver,Liver_low,Liver_high,Spleen,Spleen_low,Spleen_high
0,27291,0.055868,0.060471,0.843784,0.087951,0.068265,0.859269,0.069384,0.071347,0.749351,0.150156,0.100493
1,14517,0.018293,0.028474,0.752064,0.118384,0.129552,0.625703,0.238007,0.136290,0.724220,0.138094,0.137685
2,32987,0.020117,0.122707,0.924814,0.019277,0.055908,0.642340,0.205929,0.151731,0.722493,0.068097,0.209410
3,7904,0.004032,0.007189,0.957620,0.010469,0.031911,0.984821,0.004369,0.010811,0.965207,0.012463,0.022331
4,7904,0.021949,0.027983,0.809366,0.044353,0.146281,0.746723,0.101705,0.151572,0.837213,0.056409,0.106378
...,...,...,...,...,...,...,...,...,...,...,...,...
937,37028,0.000217,0.002526,0.873370,0.090469,0.036161,0.737942,0.108933,0.153125,0.790425,0.043780,0.165795
938,18404,0.016733,0.077987,0.746845,0.171773,0.081382,0.719815,0.172211,0.107975,0.710409,0.090854,0.198737
939,30698,0.034834,0.151994,0.796346,0.121377,0.082277,0.707737,0.171339,0.120924,0.626427,0.151621,0.221952
940,31034,0.004366,0.058384,0.819066,0.056336,0.124599,0.892052,0.069507,0.038440,0.721051,0.157274,0.121674


In [55]:
# Save the dataframe to a CSV file
predictions_df.to_csv('/kaggle/working/predictions.csv', index=False)